In [1]:
# utility
import sys
sys.path.append("..")

import utility

# data processing
import pandas as pd
import numpy as np

# modelling keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L1L2, L1, L2
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError, BinaryCrossentropy
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np
from scikeras.wrappers import KerasRegressor

# visualization
import matplotlib.pyplot as plt

In [80]:
data, labels, _ = utility.load_data()

In [81]:
train_data, train_labels, test_data, test_labels, val_data, val_labels, test_true_labels, anom_data = utility.preprocess_data(data, labels)

In [82]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=tf.shape(z_mean), mean=0., stddev=1.0)
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [83]:
def vae_loss(x, x_decoded_mean, z_log_var, z_mean):
    reconstruction_loss = tf.reduce_mean(tf.square(x - x_decoded_mean))
    kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
    kl_loss = -0.5 * tf.reduce_mean(kl_loss, axis=-1)
    vae_loss = reconstruction_loss + kl_loss
    return vae_loss

In [84]:
def build_vae(optimizer, activation, loss, dropout_rate):
    latent_dim = 2
    # encoder
    vae_input = Input(shape=(784,))
    x = Dense(256, activation='relu')(vae_input)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(rate=dropout_rate)(x)
    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)

    z = Lambda(lambda x: sampling(x))([z_mean, z_log_var])

    # decoder
    decoder_input = Input(shape=(latent_dim,))
    x = Dense(128, activation='relu')(decoder_input)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(784, activation=activation)(x)
    decoded = x

    # Encoder model
    encoder = Model(inputs=vae_input, outputs=[z_mean, z_log_var, z])

    # Decoder model
    decoder = Model(inputs=decoder_input, outputs=decoded)

    vae_output = decoder(encoder(vae_input)[2])
    vae = Model(inputs=vae_input, outputs=vae_output)

    vae.add_loss(vae_loss(vae_input, vae_output, z_log_var=z_log_var, z_mean=z_mean))

    vae.compile(optimizer=optimizer)
    return vae


In [85]:
param_grid = {
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'activation': ['relu', 'sigmoid', 'tanh'],
    'loss': [MeanSquaredError(), MeanAbsoluteError(), BinaryCrossentropy()],
    'dropout_rate': [0.0, 0.1, 0.2],
    'epochs': [100, 150, 200],
    'batch_size': [64, 128, 256, 512]
}

In [86]:
best_model = None
best_loss = float('inf')  # Initialize with a high value

for optimizer in param_grid['optimizer']:
    for activation in param_grid['activation']:
        for loss in param_grid['loss']:
            for dropout_rate in param_grid['dropout_rate']:
                for epochs in param_grid['epochs']:
                    for batch_size in param_grid['batch_size']:
                            
                        # Build the autoencoder model with current hyperparameters
                        vae = build_vae(optimizer, activation, loss, dropout_rate)
                            
                        # Train the model
                        history = vae.fit(train_data, train_data, epochs=epochs, batch_size=batch_size,
                                                  shuffle=True, validation_data=(val_data, val_data),
                                                  callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)], verbose=0)
                          
                        # Evaluate the model on validation data
                        val_loss = vae.evaluate(val_data, val_data, verbose=0)
                           
                        # Check if the current model is better than the best model
                        if val_loss < best_loss:
                            best_loss = val_loss
                            best_model = vae
                            best_params = {
                                'optimizer': optimizer,
                                'activation': activation,
                                'loss': loss,
                                'dropout_rate': dropout_rate,
                                'epochs': epochs,
                                'batch_size': batch_size
                            }

# Print the best hyperparameters and corresponding loss
print("Best Hyperparameters:")
print(best_params)
print("Best Validation Loss:", best_loss)

MemoryError: Unable to allocate 134. MiB for an array with shape (44800, 784) and data type float32

In [ ]:
# Best Hyperparameters:
# {'optimizer': 'adam', 'activation': 'tanh', 'loss': <keras.src.losses.BinaryCrossentropy object at 0x7f6862090a60>, 'dropout_rate': 0.2, 'epochs': 200, 'batch_size': 64}
# Best Validation Loss: 0.06700802594423294